# Supernova Example: BOLFI

In this example we use BOLFI by Gutmann et al. (2016) as a method of estimating cosmological parameters from supernovae data.  The following model describes the distance modulus as a function of redshift:  

$$\mu_{i}^{model}(z_{i};\Omega_{m},w_{0}) \propto 5log_{10}(\frac{c(1+z)}{h_{0}})\int_{0}^{z}\frac{dz'}{E(z')}$$


$$E(z) = \sqrt{\Omega_{m} (1+z)^{3} + (1-\Omega_{m})e^{3\int_{0}^{z} dln(1+z')[1+w(z')]}}$$

This model is used to estimate the posterior distributions of matter density $\Omega_{m}$ and the dark energy equation of state $w_{0}$.

The goal is to show how BOLFI provides useful tools to investigate on the amount of information preserved by the summary statistics.


In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
import numpy as np
import scipy.integrate
import scipy.stats as sps
from scipy.stats import skewnorm
import math

In [ ]:
import elfi

In [ ]:
from statistics import mode
from statistics import median
from statistics import mean

# Definition of the Forward Model (or simulator)

The astroabc package is coded for Python 2.6. In order to use elfi, we need to provide the forward model in Python 3.5. 

Respect to the original implementation, 1 small modification have been made:
1. rather than calling a 2-dim vector for the parameters (i.e. pars[0],pars[1]), we defined param1 and param2.

This is done in the following chunks:


In [ ]:
def read_fitres(fname):
                dtype=[('z',np.float32),('mu',np.float32),('mu_err',np.float32)]
                data=np.loadtxt(fname,skiprows = 15,dtype =dtype, usecols=(34,48,50))
                return data

def bin_data(z,mu,err,Nbins=10):
    binw = (z.max()-z.min())/Nbins
    #print binw
    zbins = np.arange(z.min(),z.max(),binw)
    mu_in_bin=[];err_in_bin=[]
    avmu_bin=[] ; averr_bin=[]
    for i in range(Nbins):
        mu_in_bin.append([]); err_in_bin.append([])
    for i,m in enumerate(mu):
        for j in range(Nbins): 
            if z[i] >=z.min()+ j*binw and z[i] < z.min() + (j+1)*binw:
                mu_in_bin[j].append(m) ; err_in_bin[j].append(err[i])
    for i in range(Nbins):
        if mu_in_bin[i]:
            avmu_bin.append(np.mean(mu_in_bin[i]))
        else: avmu_bin.append(0)
        if err_in_bin[i]:
            averr_bin.append(np.mean(err_in_bin[i]))
        else: averr_bin.append(0)
    return  zbins, avmu_bin,averr_bin,mu_in_bin,mu_in_bin

def read_data():
    data = read_fitres("example_data_file.txt")

    #Keep data points with z>0.5
    z_new = data['z'][0]
    mu_new = data['mu'][0]
    err_new = data['mu_err'][0]

    for i in range(1,345):
        if data['z'][i] >= 0.5:
            z_new = np.append(z_new,data['z'][i])
            mu_new = np.append(mu_new,data['mu'][i])
        err_new = np.append(err_new,data['mu_err'][i])

    #bin this data
    zbins,avmu_bin,averr_bin,mu_in_bin_new,mu_in_bin_new = bin_data(z_new,mu_new,err_new,Nbins=20)
    return zbins,avmu_bin,averr_bin,mu_in_bin_new,mu_in_bin_new




In [ ]:
c_km_per_s = 299792.458

class DistanceCalc(object):
    def __init__(self,om,ok,ol,wmodel,de_params,h0):
        '''
        om = omega_matter
        ok = omega curvature
        ol = omega lambda
        wmodel= -1 for LCDM(w0=-1), 0 for (w0,wa) parametrisation, 1 for w0,wa,ap parametrization, 2 for early dark energy
        de_params = -1 (if wmodel=-1), =[w0,wa] (if wmodel==0 )
        h0 = hubble constant e.g. 0.7
        '''
        self.om=om
        self.ol=ol
        self.ok=ok
        if self.om + self.ol ==1.: self.is_flat = True
        self.wmodel=wmodel
        self.de_params=de_params
        self.h0=h0
        self.d_h =  c_km_per_s/(100.) #Mpc/h

    def wfunc(self,a):
        if self.wmodel == -1:
            w0 = self.de_params
            return w0
        elif self.wmodel == 0: # e.g. Linder 2003
            w0,wa=self.de_params
            return w0 + (1.0-a)*wa
        elif self.wmodel == 1: # e.g Huterer & Turner 2001
            w0,wa,ap=self.de_params
            return w0 + (ap-a)*wa
        elif self.wmodel ==2: # Wetterich 2004
            w0,ode_e = self.de_params
            b = -3.*(  w0/( np.log((1.-ode_e)/ode_e) + np.log((1.-self.om)/self.om)  ))
            return w0/(1.0 - b*(np.log(a)))
            

    def w_integrand(self,a):
        return (1.+ self.wfunc(a))/a

    def w_int(self,z):
        a=1./(1.+z)
        return 3.0*scipy.integrate.quad(self.w_integrand,a,1.0,epsrel=1e-6,limit=50)[0]

    def e_z_inverse(self,z):
        if self.wmodel == -1:
            return 1./(np.sqrt(self.om*(1+z)**3 + self.ok*(1+z)**2 + self.ol))
        else:
            return 1./(np.sqrt(self.om*(1+z)**3 + self.ok*(1+z)**2 + self.ol*np.exp(self.w_int(z))))

    def d_c(self,z):
        return self.d_h*scipy.integrate.quad(self.e_z_inverse,0.0,z,epsrel=1e-6,limit=50)[0]


    def d_m(self,z):
        '''returns los comoving distance in Mpc/h '''
       	return self.d_c(z)

    def hubble(self,z):
        return 100.*self.h0*1./self.e_z_inverse(z)

    def d_l(self,z):
        '''luminosity dist in Mpc '''
        return (1.+z)*self.d_m(z)/self.h0

    def d_a(self,z):
        '''Ang diameter distance in Mpc'''
        return self.d_m(z)/(1.+z)/self.h0

    def mu(self,z):
        'distance modulus'''
        return 5.*np.log10(self.d_l(z)*1.E6/10.) 



Definition of the forward model.  This must be able to simulate the data at every point in parameter space.  At each $z_{i}$ the simulation uses $\mu_{model}(z_{i};\Omega_{m},w_{0})$ given by
$$\mu_{i}^{model}(z_{i};\Omega_{m},w_{0}) \propto 5log_{10}(\frac{c(1+z)}{h_{0}})\int_{0}^{z}\frac{dz'}{E(z')}$$

to produce a value of distance modulus.  To account for noise in the data we then add a number randomly drawn from a skewed normal distribution.  

In [ ]:
def model(param1,param2, batch_size=1, random_state=None): #param = [om, w0] 
    if param1 < 0.0 or param1 > 1.0:
        return [None]*len(zbins)
    else:
        model_1_class = DistanceCalc(param1,0,1-param1,0,[param2,0],0.7)  #om,ok,ol,wmodel,de_params,h0
        data_abc = np.zeros(len(zbins))
        for i in range(len(zbins)):
                data_abc[i] = model_1_class.mu(zbins[i]) + skewnorm.rvs(a, loc=e, scale=w, size=1)
        return data_abc

# The Data

First, we need to provide a dataset.  The aim is to generate values of the distance modulus, $\mu_{i}$, with fixed "true" parameters $\Omega_{m}$ and $w_{0}$

$$\Omega_{m} = 0.3$$
$$w_{0} = -1.0$$

SNANA is used to generate ~400 supernova light curves which are then fit with the SALT-II light curve fitter. The  data span the redshift interval $z \in [0.5,1.0]$ and are binned into 20 redshift bins.

In [ ]:
zbins,avmu_bin,averr_bin,mu_in_bin_new,mu_in_bin_new = read_data()

### Adding noise to the data

To add artificial noise to the data we use a skewed normal distribution.  The standard normal distribution has a probability distribution function given by

$$ \phi(x) = \frac{1}{\sqrt{2 \pi}} e^{-\frac{x^{2}}{2}}$$

and a cumulative distribution function: 

$$\Phi(x) = \frac{1}{2} [1 + erf(\frac{x}{\sqrt{2}})] $$

The skewed normal distribution $f(x)$ with parameter $\alpha$ is given by 

$$f(x) = 2\phi(x)\Phi(\alpha x)$$

Using this probability distribution function, we can draw a random sample from the skewed normal distribution.

In [ ]:
e = -0.1 #location
w = 0.3 #scale
a = 5.0 #skew

plt.figure(figsize=(17,8))
plt.hist(skewnorm.rvs(a, loc=e, scale=w, size=10000),normed=True,bins=20,color='#593686')
plt.title("Distribution of a random sample",fontsize=17);

From this distribution, the noisy data is generated.  At each $z_{i}$ a random number is drawn from the above distribution and added to $\mu_{i}$.  

In [ ]:
data = np.zeros(len(zbins)) 

for i in range(len(zbins)):
    data[i] = avmu_bin[i] + skewnorm.rvs(a, loc=e, scale=w, size=1)

A comparison of the data before and after noise is added is shown below:

In [ ]:
plt.figure(figsize=(17,8))
plt.errorbar(zbins,avmu_bin,averr_bin,marker="o",linestyle="None",label="without noise",color='#593686')
plt.scatter(zbins,data,color='r',label="with noise")
plt.legend(loc="upper left",prop={'size':17});
plt.xlabel("$z$",fontsize=20)
plt.ylabel("$\mu(z)$",fontsize=20)
plt.title("Data before and after noise is added",fontsize=17);

To demonstrate the non-Gaussian distribution of the data at each $z_{i}$, we focus on the data at $z_{1}=0.5$.  The distribution of a large random sample at this redshift is shown below.  Each value in this sample is generated by adding a randomly drawn number from the skewed normal distribution to $\mu_{1}$.  The value of $\mu_{1}$ before noise is added is shown in red. As we can see the data is now a skewed distribution around the expected mean.

In [ ]:
z = 0
distribution = np.zeros(10000)

for j in range(10000):
    distribution[j] = avmu_bin[z] + skewnorm.rvs(a, loc=e, scale=w, size=1)

plt.figure(figsize=(17,8))
plt.title("Distribution of the data at redshift z=0.5",fontsize=17);
plt.hist(distribution,bins=20,color='#593686',normed=True)
plt.plot((avmu_bin[z], avmu_bin[z]), (0, 2.5), 'r-', label="True $\mu$ at $z = 0.5$");
plt.legend(prop={'size':16});

In [ ]:
###call of the forward model
model(0.7,0.5)

In [ ]:
###data used by Jennings et al.
data = np.load('dataSupernorva.npy')

In [ ]:
###data simulated
data = np.loadtxt(fname = "data_moduli.txt")

# Definition of the quantities to run BOLFI

## Prior distributions of $\Omega_{m}$ and $w_{0}$

The prior distributions of $\Omega_{m}$ and $w_{0}$ are chosen as follows:
* $\Omega_{m}$: we use a Beta distribution.
* $w_{0}$: we use a normal distribution.

Note that because of the constraint on the forward model, on $\Omega_{m} \in (0,1)$, rather than using a Normal distribution (Jennings et al. 2016) we used a Beta Distribution. This is done for not leading to errors when calling BOLFI.

In [ ]:
#p1 = elfi.Prior('normal', 0.3, 0.1)
#p1 = elfi.Prior('truncnorm', 0, 1)
p1 = elfi.Prior('beta',3,3)
p2 = elfi.Prior('normal', -0.5, 0.5)

In [ ]:
model=elfi.tools.vectorize(model)

## Distance function

In this example the metric $\rho$ defined by Jennings et al. 2016 is:
$$\rho(\mu,\mu_{sim}(z)) = \sum_{i} \frac{(\mu_{i} - \mu_{sim}(z_{i}))^{2}}{2 \sigma_{i}^{2}}$$
where $\sigma_{i}$ is the error on the data point $\mu_{i}$.

We do not consider the denominator.

In [ ]:
def my_dist(b,a):
        #return np.array([np.sqrt(np.sum((a-b)**2/(2*np.array(averr_bin)**2)))])
        return np.array([np.sum(((a-b)/averr_bin)**2)])

In [ ]:
my_dist(elfi.Simulator.generate(Y),elfi.Simulator.generate(Y))
#my_dist(data,data)

With BOLFI, other several distance metrics could be selected (look at scipy.distances)

In [ ]:
d = elfi.Distance(my_dist, Y)

elfi.Simulator.generate(d)

In [ ]:
log_d = elfi.Operation(np.log, d)
elfi.Simulator.generate(log_d)

### BOLFI

In [ ]:
bolfi = elfi.BOLFI(log_d, batch_size=1, initial_evidence=500, update_interval=1,
                   bounds={'p1':(0, 1), 'p2':(-3, 0)}, acq_noise_var=[1, 1])

In [ ]:
%time post = bolfi.fit(n_evidence=2000)

In [ ]:
bolfi.target_model

In [ ]:
bolfi.plot_state();

In [ ]:
bolfi.plot_state()
plt.xlim(0, 1)
plt.ylim(-3, 0)
for idx, ax in enumerate(plt.gcf().axes):
    #ax.add_artist(plt.Circle((0,0), 47.9, color='k', fill=False, linestyle='--'))
    ax.axvline(0.3, color='red', linestyle='--')
    ax.axhline(-1, color='red', linestyle='--')
    if idx == 1:
        ax.set_visible(False)
plt.savefig('target_surface_GP.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
bolfi.plot_discrepancy();

In [ ]:
# Running the Metropolis-Hastings: remember to perform the thinning! (Done in R)
n_samples=10000
n_chains=4
%time result_BOLFI = bolfi.sample(n_samples, n_chains=n_chains, algorithm='metropolis')

### BOLFI RESULTS

In [ ]:
result_BOLFI

In [ ]:
result_BOLFI.plot_traces();
#plt.savefig('traceplotsM-H.png', dpi=150, bbox_inches='tight')

In [ ]:
result_BOLFI.plot_pairs()
#plt.savefig('pairsM-H.png', dpi=150, bbox_inches='tight')

In [ ]:
import seaborn as sb

In [ ]:
f = sb.jointplot(x="p1", y="p2", ylim=(-2,-0.25), data=result_BOLFI.samples, kind="kde")

f.ax_marg_x.axvspan(left_x, right_x, color='blue', alpha=0.15)  #, linestyle='-')
f.ax_marg_y.axhspan(left_y, right_y, color='blue', alpha=0.15)

f.ax_marg_x.axvspan(mean(result_BOLFI.samples['p1'])-np.sqrt(np.var(result_BOLFI.samples_array, axis=0))[0],mean(result_BOLFI.samples['p1'])+np.sqrt(np.var(result_BOLFI.samples_array, axis=0))[0], color='yellow',alpha=0.15)
f.ax_marg_y.axhspan(mean(result_BOLFI.samples['p2'])-np.sqrt(np.var(result_BOLFI.samples_array, axis=0))[1],mean(result_BOLFI.samples['p2'])+np.sqrt(np.var(result_BOLFI.samples_array, axis=0))[1], color='yellow',alpha=0.15)


f.ax_marg_x.axvline(0.3, color='red', linestyle='--')
f.ax_marg_y.axhline(-1, color='red', linestyle='--')
f.ax_joint.axvline(0.3, color='red', linestyle='--')
f.ax_joint.axhline(-1, color='red', linestyle='--', label='Truth')
f.set_axis_labels(r'$\Omega_m$', r'$\omega_0$')
f.ax_joint.plot(result_BOLFI.sample_means['p1'], result_BOLFI.sample_means['p2'], 'gD',label='BOLFI mean')
f.ax_joint.plot(0.3,-1.03, 'bo',label='CSL, G=1')
#f.ax_joint.plot(mode(result_BOLFI.samples['p1']), mode(result_BOLFI.samples['p2']), 'yX', label='BOLFI mode')
#f.ax_joint.plot(median(result_BOLFI.samples['p1']), median(result_BOLFI.samples['p2']), 'co', label='BOLFI median')
f.ax_joint.plot(0.297,-1.112, 'yX', label='ABC-SMC Jennings')
f.ax_joint.legend(loc='upper right', frameon=True) #ncol=2)
f.savefig('contour_posterior_figure.png', bbox_inches='tight', dpi=150)

In [ ]:
np.save("BOLFI_output_p1",result_BOLFI.samples['p1'])

In [ ]:
np.save("BOLFI_output_p2",result_BOLFI.samples['p2'])

In [ ]:
result_BOLFI.sample_means_summary

In [ ]:
np.sqrt(np.var(result_BOLFI.samples_array, axis=0))